<a href="https://colab.research.google.com/github/miquelt9/HackUPC23_HP/blob/main/HPForecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libraries import

In [3]:
pip install skforecast

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.8/343.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
      Successfully uninstalled tqdm-4.65.0


In [35]:
# Data manipulation
# ==============================================================================
import numpy as np
import pandas as pd
import math

# Plots
# ==============================================================================
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
%matplotlib inline

# Modeling and Forecasting
# ==============================================================================
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# from skforescast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster

# Warnings configuration
# ==============================================================================
import warnings
# warnings.filterwarnings('ignore')

Data download, functions,and prep

In [ ]:
# Data download
# ==============================================================================
url = 'https://raw.githubusercontent.com/miquelt9/HackUPC23_HP/main/train.csv'
data = pd.read_csv(url, sep=',')

In [50]:
  # Checking
  sale_inv = list(zip(data['sales_units'],data['inventory_units']))
  non_sales = 0
  non_inv = 0
  non_non = 0
  for i in range(len(sale_inv)):
    (sales, inv) = sale_inv[i]
    if math.isnan(sales) and math.isnan(inv):
      non_non += 1
    elif math.isnan(inv):
      non_inv += 1
    elif math.isnan(sales):
      non_sales += 1


  print("NaN Nothing: " + str(non_non))
  print("NaN inventory: " + str(non_inv))
  print("NaN sales: " + str(non_sales))

NaN Nothing: 8
NaN inventory: 485
NaN sales: 483


In [52]:
def fill_gaps(data):
  sale_inv = list(zip(data['sales_units'],data['inventory_units']))
  for i in range(len(sale_inv)):

    (act_sales, act_inv) = sale_inv[i]
    if i > 0: 
      (last_sales, last_inv) = sale_inv[i-1]
    if i < len(sale_inv)-1: 
      (next_sales, next_inv) = sale_inv[i+1]

    if math.isnan(act_sales) and math.isnan(act_inv):
      act_inv = (last_inv+next_inv)/2
    if math.isnan(act_inv):
      act_inv = last_inv - act_sales
      sale_inv[i] = (act_sales, act_inv)
    elif math.isnan(act_sales):
      act_sales = last_inv - act_inv
      sale_inv[i] = (act_sales, act_inv)
    
  return data

def sum_same_yearweek(data):
  

In [103]:
url = 'https://raw.githubusercontent.com/miquelt9/HackUPC23_HP/main/train.csv'
data = pd.read_csv(url, sep=',')

# Data preparation
# ==============================================================================
# Format the date
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
# Convert year_week into weeks 0-max
data['year'] = round(data['year_week']/100)
first_year = min(data['year'])
data['week'] = ((data['year_week']/100)%1)*100

data['week'] = data['week'] + (52*(data['year']-first_year))

data['product_number'] = list(map(int, data['product_number']))

# # Droping columns
data = data.drop('display_size', axis=1)
data = data.drop('specs', axis=1)

data = fill_gaps(data)
print("-----> There are " + str(len(data)) + " columns before grouping\n")
data = data.groupby(['year_week','product_number']).sum()
print("-----> There are " + str(len(data)) + " columns AFTER grouping\n")

grouped = data.groupby('product_number') #grouped['product_number']
grouped_list = [grouped.get_group(x) for x in grouped.groups]

for prod in grouped:
  print(grouped_list[0])

-----> There are 15454 columns before grouping

-----> There are 7709 columns AFTER grouping

                          reporterhq_id  sales_units  inventory_units    year  \
year_week product_number                                                        
202119    6909                       93          2.0             35.0  2021.0   
202120    6909                       93          0.0             70.0  2021.0   
202121    6909                       93          3.0            137.0  2021.0   
202122    6909                       93          0.0            274.0  2021.0   
202123    6909                       93          0.0            333.0  2021.0   
...                                 ...          ...              ...     ...   
202314    6909                      108          0.0            109.0  4046.0   
202315    6909                      108          1.0            159.0  4046.0   
202316    6909                      108         34.0            113.0  4046.0   
202317    6909 

<ipython-input-103-290e34cebb84>:23: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = data.groupby(['year_week','product_number']).sum()


                          reporterhq_id  sales_units  inventory_units    year  \
year_week product_number                                                        
202119    6909                       93          2.0             35.0  2021.0   
202120    6909                       93          0.0             70.0  2021.0   
202121    6909                       93          3.0            137.0  2021.0   
202122    6909                       93          0.0            274.0  2021.0   
202123    6909                       93          0.0            333.0  2021.0   
...                                 ...          ...              ...     ...   
202314    6909                      108          0.0            109.0  4046.0   
202315    6909                      108          1.0            159.0  4046.0   
202316    6909                      108         34.0            113.0  4046.0   
202317    6909                      108        143.0             90.0  4046.0   
202318    6909              

In [ ]:
steps = 36
data_train = data[:-steps]
data_test  = data[-steps:]

print(f"Train dates : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})")
print(f"Test dates  : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})")

fig, ax = plt.subplots(figsize=(7, 3))
data_train['y'].plot(ax=ax, label='train')
data_test['y'].plot(ax=ax, label='test')
ax.legend();


In [ ]:
# Create and train forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                regressor = RandomForestRegressor(random_state=123),
                lags      = 6
             )

forecaster.fit(y=data_train['y'])
forecaster